In [1]:
import os
import pickle
import pandas as pd
import numpy as np
import nibabel as nib
import pcntoolkit as ptk 
from pcntoolkit.dataio.fileio import load as ptkload
from pcntoolkit.dataio.fileio import save as ptksave
from pcntoolkit.dataio.fileio import save_nifti, load_nifti
from pcntoolkit.normative import predict, evaluate
from pcntoolkit.util.utils import compute_MSLL, create_design_matrix, calibration_descriptives

from scipy import stats

from matplotlib import pyplot as plt
import seaborn as sns
import pingouin as pg

#%% LOAD IN DATA AND MASK BY AVAILABLE PARTICIPANTS
# Data IO and generation

# globals
root_dir = '/project_cephfs/3022017.06/ENIGMA_ANX/'
proc_dir = os.path.join(root_dir,'Z_stat/')
data_dir = os.path.join(proc_dir,'data/')
vox_dir = os.path.join(proc_dir,'vox/')
w_dir = os.path.join(vox_dir,'NPM_comparisons/Diagnoses/')
mask_nii = ('/opt/fmriprep/templateflow/tpl-MNI152NLin2009cAsym/tpl-MNI152NLin2009cAsym_res-02_desc-brain_mask.nii.gz')
ex_nii = os.path.join(data_dir, 'ENIGMA_FC_tr_1.nii.gz')

/home/preclineu/hansav/.conda/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/preclineu/hansav/.conda/envs/py38/lib/python3.8/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
# Load in the Z_est file
Z_est_clinical = ptkload(os.path.join(vox_dir,'Z_predcl.pkl'), mask=mask_nii)

In [3]:
#Load in the diagnosis information
metadata_cl_diagnosis = pd.read_csv('/project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/data/clinical_te.csv')
Diagnoses_info = pd.read_csv('/project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/data/all_test_validation.csv', usecols = ['SubjectID_ENIGMA', 'Principal_diagnosis_current'] )
merged_clinical_test_data = pd.merge(metadata_cl_diagnosis, Diagnoses_info, on='SubjectID_ENIGMA', how='left')
merged_clinical_test_data['Principal_diagnosis_current'] = merged_clinical_test_data['Principal_diagnosis_current'].replace({np.nan: 'NA', 'nan': 'NA'})



In [4]:
print(len(Z_est_clinical))
print(len(metadata_cl_diagnosis))
print(len(merged_clinical_test_data))

581
581
581


In [5]:
#split the Z_est clinical into the control and patients  ***and remove other and schizophrenia as diagnosis are not of interest***
mask_Diganosis = merged_clinical_test_data['Healthy_or_patient'].eq(1)
mask_exclude_diagnosis = ~merged_clinical_test_data['Principal_diagnosis_current'].isin(['others', 'schizophrenia'])
 #remove other and schizophrenia as not primary affective conditions 
combined_Diagnosis_mask = mask_Diganosis & mask_exclude_diagnosis

print(len(mask_Diganosis))
print(len(mask_exclude_diagnosis))
print(len(combined_Diagnosis_mask))


Diagnoses_info = merged_clinical_test_data[combined_Diagnosis_mask]
Affective_sample_deviations = Z_est_clinical[combined_Diagnosis_mask]

581
581
581


In [6]:
print(len(Affective_sample_deviations))
print(len(Diagnoses_info))
Primary_Diagnoses = np.unique(merged_clinical_test_data['Principal_diagnosis_current'])
Primary_Diagnoses = [diagnosis for diagnosis in Primary_Diagnoses if diagnosis not in ['NA', 'others', 'schizophrenia']]
print('Diagnoses that will be analysed: ', Primary_Diagnoses)

222
222
Diagnoses that will be analysed:  ['gad', 'major depressive disorder', 'ocd', 'panic disorder', 'ptsd', 'sad', 'specific phobia']


In [7]:
for d in Primary_Diagnoses:
    mask_Diganosis = Diagnoses_info['Principal_diagnosis_current'].eq(d)
    print(sum(mask_Diganosis == True), d)
    Z_est_Diganosis = Affective_sample_deviations[mask_Diganosis]
    Z_est_Diganosis_trans = np.transpose(Z_est_Diganosis)
    filename_part = (w_dir +'Z_est_'+ d.replace(' ', '_') +'.nii.gz')
    print(filename_part)
    save_nifti(Z_est_Diganosis_trans, filename_part, examplenii=ex_nii, mask=mask_nii, dtype='float32')

48 gad
/project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/NPM_comparisons/Diagnoses/Z_est_gad.nii.gz
11 major depressive disorder
/project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/NPM_comparisons/Diagnoses/Z_est_major_depressive_disorder.nii.gz
68 ocd
/project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/NPM_comparisons/Diagnoses/Z_est_ocd.nii.gz
2 panic disorder
/project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/NPM_comparisons/Diagnoses/Z_est_panic_disorder.nii.gz
55 ptsd
/project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/NPM_comparisons/Diagnoses/Z_est_ptsd.nii.gz
31 sad
/project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/NPM_comparisons/Diagnoses/Z_est_sad.nii.gz
7 specific phobia
/project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/NPM_comparisons/Diagnoses/Z_est_specific_phobia.nii.gz


In [8]:
# THRESHOLD THE NII FILES USING FSLMATHS
for d in Primary_Diagnoses:
    in_filename = (w_dir +'Z_est_'+d.replace(' ', '_') +'.nii.gz')
    #print(filename)
    out_filename_pos = (w_dir +'Z_est_' +d.replace(' ', '_')  +'_pos2pt6.nii.gz')
    out_filename_neg = (w_dir +'Z_est_' +d.replace(' ', '_') +'_neg2pt6.nii.gz')
    #print(out_filename_pos, out_filename_neg)
    
    command = ('fslmaths ' +in_filename +' -thr 2.6 ' +out_filename_pos)
    print(command)
    os.system(command)
    !command
    
    command = ('fslmaths ' +in_filename +' -mul -1 -thr 2.6 ' +out_filename_neg)
    print(command)
    os.system(command)
    !command

fslmaths /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/NPM_comparisons/Diagnoses/Z_est_gad.nii.gz -thr 2.6 /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/NPM_comparisons/Diagnoses/Z_est_gad_pos2pt6.nii.gz
fslmaths /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/NPM_comparisons/Diagnoses/Z_est_gad.nii.gz -mul -1 -thr 2.6 /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/NPM_comparisons/Diagnoses/Z_est_gad_neg2pt6.nii.gz
fslmaths /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/NPM_comparisons/Diagnoses/Z_est_major_depressive_disorder.nii.gz -thr 2.6 /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/NPM_comparisons/Diagnoses/Z_est_major_depressive_disorder_pos2pt6.nii.gz
fslmaths /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/NPM_comparisons/Diagnoses/Z_est_major_depressive_disorder.nii.gz -mul -1 -thr 2.6 /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/NPM_comparisons/Diagnoses/Z_est_major_depressive_disorder_neg2pt6.nii.gz
fslmaths /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vo

In [9]:
#BINARISE THE NII FILES USING FSLMATHS
for d in Primary_Diagnoses:
    in_filename_pos = (w_dir +'Z_est_' +d.replace(' ', '_') +'_pos2pt6.nii.gz')
    in_filename_neg = (w_dir +'Z_est_' +d.replace(' ', '_') +'_neg2pt6.nii.gz')
    #print(filename)
    out_filename_pos = (w_dir +'Z_est_' +d.replace(' ', '_') +'_pos2pt6_bin.nii.gz')
    out_filename_neg = (w_dir +'Z_est_' +d.replace(' ', '_') +'_neg2pt6_bin.nii.gz')
    #print(out_filename_pos, out_filename_neg)
    
    command = ('fslmaths ' +in_filename_pos +' -bin  ' +out_filename_pos)
    print(command)
    os.system(command)
    !command
    
    command = ('fslmaths ' +in_filename_neg +' -bin  ' +out_filename_neg)
    print(command)
    os.system(command)
    !command 

fslmaths /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/NPM_comparisons/Diagnoses/Z_est_gad_pos2pt6.nii.gz -bin  /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/NPM_comparisons/Diagnoses/Z_est_gad_pos2pt6_bin.nii.gz
fslmaths /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/NPM_comparisons/Diagnoses/Z_est_gad_neg2pt6.nii.gz -bin  /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/NPM_comparisons/Diagnoses/Z_est_gad_neg2pt6_bin.nii.gz
fslmaths /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/NPM_comparisons/Diagnoses/Z_est_major_depressive_disorder_pos2pt6.nii.gz -bin  /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/NPM_comparisons/Diagnoses/Z_est_major_depressive_disorder_pos2pt6_bin.nii.gz
fslmaths /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/NPM_comparisons/Diagnoses/Z_est_major_depressive_disorder_neg2pt6.nii.gz -bin  /project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/vox/NPM_comparisons/Diagnoses/Z_est_major_depressive_disorder_neg2pt6_bin.nii.gz
fslmaths /project_cephfs/3022017.06/

In [10]:
#%%SUM ACROSS PARTICIPANTS
for d in Primary_Diagnoses:
    in_filename_pos = (w_dir +'Z_est_' +d.replace(' ', '_') +'_pos2pt6_bin.nii.gz')
    in_filename_neg = (w_dir +'Z_est_' +d.replace(' ', '_') +'_neg2pt6_bin.nii.gz')
    thresh_bin_nii_pos = load_nifti(in_filename_pos, mask=mask_nii)
    thresh_bin_nii_neg = load_nifti(in_filename_neg, mask=mask_nii)
    
    out_filename_pos = (w_dir +'Z_est_' +d.replace(' ', '_') +'_count_pos.nii.gz')
    out_filename_neg = (w_dir +'Z_est_' +d.replace(' ', '_') +'_count_neg.nii.gz')
    #print(out_filename_pos, out_filename_neg)
    
    #Sum along the 2nd dimension(4D = volumes = participants) of the Positive thresholded, binarised file
    sum_diagnosis_pos = np.sum(thresh_bin_nii_pos,axis=1)
    #Save as nii
    save_nifti(sum_diagnosis_pos, out_filename_pos, examplenii=ex_nii, mask=mask_nii, dtype='float32')
    
    #Sum along the 2nd dimension(4D = volumes = participants) of the Negative thresholded, binarised file
    sum_diagnosis_neg = np.sum(thresh_bin_nii_neg,axis=1)
    #Save as nii
    save_nifti(sum_diagnosis_neg,out_filename_neg, examplenii=ex_nii, mask=mask_nii, dtype='float32')
    